In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/final_data

/content/drive/MyDrive/final_data


In [4]:
import requests
import json
import pandas as pd

## 기본 세팅 부분
headers = {"X-Naver-Client-Id": "lJrKhZKFXCGXSoYzh7f7",
           "X-Naver-Client-Secret":"YYPQvTkBxR"}

url = "https://openapi.naver.com/v1/datalab/shopping/categories"
## 대분류 카테고리 설정
categories = [
      {"name": "가구/인테리어", "param": [ "50000004"]},
      {"name": "도서", "param": [ "50005542"]},
      #{"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "생활/건강", "param": [ "50000008"]},
      {"name": "스포츠/레저", "param": [ "50000007"]},
      {"name": "식품", "param": [ "50000006"]},
      {"name": "출산/육아", "param": [ "50000005"]},
      #{"name": "패션의류", "param": [ "50000000"]},
      {"name": "패션잡화", "param": [ "50000001"]},
      {"name": "화장품/미용", "param": [ "50000002"]}
  ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
data = {
  "startDate": "2021-05-10",
  "endDate": "2023-05-31",
  "timeUnit": "month",
  "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "패션의류", "param": [ "50000000"]},
  ],
  "device":"",
  "gender":"",
  "ages":[]
}
## 데이터 정제 및 merge 파트

def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df

response = requests.post(url, headers=headers, json=data)

results = response.json()["results"]


df_a = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
df_b = change_index(pd.DataFrame(results[1]["data"]), "패션의류")
##
df_all = pd.merge(df_a, df_b, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
for category in categories:
    if len(data["category"]) > 2:
        data["category"].pop(2)
    data["category"].append(category)

    result = requests.post(url, headers=headers, json=data)

    results = result.json()["results"]

    df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

    df_all = pd.merge(df_all, df, left_on="period", right_on="period")

## 엑셀에 저장
df_all.to_excel("data_sample.xlsx", sheet_name="click ratio")
df_all.head()

,디지털/가전,패션의류,가구/인테리어,도서,생활/건강,스포츠/레저,식품,출산/육아,패션잡화,화장품/미용
period,,,,,,,,,,
2021-05-01,15.06171,13.32508,6.65428,0.13384,16.87566,9.24052,7.69353,6.71983,13.55433,3.87216
2021-06-01,15.65489,13.08888,6.37452,0.27189,16.49478,9.53649,8.02306,5.96851,13.28828,3.75099
2021-07-01,18.97638,12.28260,6.99978,0.37126,17.79258,9.81018,9.00838,5.65315,12.72383,4.11032
2021-08-01,18.78373,10.09543,7.28965,0.39691,18.19988,9.46159,9.78885,5.18658,11.10570,4.25004
2021-09-01,17.87758,12.27366,7.03428,0.38985,17.28150,9.66908,10.18205,5.74338,11.38217,3.99905
